# Data Gathering
## Import Libraries & Tools

In [47]:
import pandas as pd
import numpy as np
import time
import glob
import re
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from datetime import date

## Create Directory Structure

In [48]:
DATA_DIR = os.path.join(
    os.path.dirname(os.path.realpath("__file__")), "data"
)

BOULDER_MEN_DIR            = os.path.join(DATA_DIR, "Boulder/Men")
BOULDER_WOMEN_DIR          = os.path.join(DATA_DIR, "Boulder/Women")
LEAD_MEN_DIR               = os.path.join(DATA_DIR, "Lead/Men")
LEAD_WOMEN_DIR             = os.path.join(DATA_DIR, "Lead/Women")
SPEED_MEN_DIR              = os.path.join(DATA_DIR, "Speed/Men")
SPEED_WOMEN_DIR            = os.path.join(DATA_DIR, "Speed/Women")
COMBINED_MEN_DIR           = os.path.join(DATA_DIR, "Combined/Men")
COMBINED_WOMEN_DIR         = os.path.join(DATA_DIR, "Combined/Women")
BOULDER_AND_LEAD_MEN_DIR   = os.path.join(DATA_DIR, "Boulder & Lead/Men")
BOULDER_AND_LEAD_WOMEN_DIR = os.path.join(DATA_DIR, "Boulder & Lead/Women")

dirs = [BOULDER_MEN_DIR, BOULDER_WOMEN_DIR, LEAD_MEN_DIR, LEAD_WOMEN_DIR,
       SPEED_MEN_DIR, SPEED_WOMEN_DIR, COMBINED_MEN_DIR, COMBINED_WOMEN_DIR,
       BOULDER_AND_LEAD_MEN_DIR, BOULDER_AND_LEAD_WOMEN_DIR]

# Create directory if it doesn't exist
for dir in dirs:
    if not os.path.exists(dir):
        os.makedirs(dir)
        
# File to store names of events that have already been scraped
try:
    ALREADY_SCRAPED = os.path.join(DATA_DIR, "scraped_events.txt")
    # Create file
    with open(ALREADY_SCRAPED, 'x') as fp:
        pass
except:
    if os.stat(ALREADY_SCRAPED).st_size == 0:
        print("No data has been scraped yet!")

## IFSCScraper Class Definition

In [165]:
class IFSCScraper():
    """
    Define a class for the scraper that will be used to gather data from the IFSC website
    (ifsc-climbing.org)
    Includes methods that allow for scraping different pages and different information
    """
    # Page url
    url  = 'https://www.ifsc-climbing.org/index.php/world-competition/last-result'
    url2 = 'https://ifsc.results.info/#/athlete/'
    
    def __init__(self, debug=False):
        """
        Initialize a scraper object with its own browser instance
        Input:
            debug - Indicates whether this is a debug instance for quicker development
        """
        self.debug = debug
        self.year_events = {}

        self.generate_driver()            
        time.sleep(1)
    
    def generate_driver(self):
        """
        Initialize Selenium web browser
        Input:
            N/A
        """
        try:
            self.driver = webdriver.Firefox()
        except:
            print('Error: Could not create WebDriver object...')
        
    def load_page(self, link, athlete_page=0, timeout=10, wait_after=1):
        """
        Helper function that loads a page and waits for timeout
        input:
            link - Link to the page we wish to load
            timeout - Seconds to wait before timing out
            wait_after - Seconds to wait after loading
        output:
            N/A
        """

        # Visit link
        self.driver.get(link)
        wait = WebDriverWait(self.driver, timeout)

        # Attempt to open link
        try:
            if athlete_page:
                wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@class='athlete-info left-side']")))
            else:
                wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@class='uk-container']")))
        except TimeoutException:
            print("Timed out waiting for page " + link + " to load")
            self.driver.quit()

        # Wait for page to load
        time.sleep(wait_after)
        
    def get_comp_years_and_league(self):
        """
        Parse the world-competition/last-result page to find and return years and leagues
        input:
            N/A
        output:
            List of tuples containing comp year and league
        """
        year_league_comb = []
        
        try:
            self.load_page(IFSCScraper.url)
        except:
            print('Error loading page!')
            
        # The dropdown menus to pick years/leagues/events is in an iframe - we need to switch to it
        frame = self.driver.find_element(By.XPATH, '/html/body/div[1]/div[4]/div/iframe')
        self.driver.switch_to.frame(frame)

        # Dropdown menus for each choice
        year_dd     = self.driver.find_element(By.XPATH, '//select[@id="years"]')
        league_dd   = self.driver.find_element(By.XPATH, '//select[@id="indexes"]')

        # Select all options for 'Year' and 'League' dropdown menus
        year_opts = Select(year_dd).options
        league_opts = Select(league_dd).options

        # Extract text of each of the above options
        years   = [opt.text for opt in year_opts]
        # leagues = [opt.text for opt in league_opts[1:]] # all leagues
        league = league_opts[1].text #world cup only for now

        year_league_comb = []
        for year in years:
            if (year, league) not in year_league_comb:
                year_league_comb.append((year, league))

        return year_league_comb

    def get_years_events(self, years):
        """
        Iterate through each year and get that years events
        input:
            N/A
        output:
            List of tuples containing comp year and league
        """
        try:
            self.load_page(IFSCScraper.url)
        except:
            print('Error loading page!')
        
        # The dropdown menus to pick years/leagues/events is in an iframe - we need to switch to it
        frame = self.driver.find_element(By.XPATH, '/html/body/div[1]/div[4]/div/iframe')
        self.driver.switch_to.frame(frame)

        # Dropdown menus for each choice
        year_dd, league_dd, event_dd, cat_dd = self.get_dropdowns(self.driver)
                
        # Creating wait
        wait = WebDriverWait(self.driver, 10)
        
        # Iterate through years provided by get_comp_years_and_league
        for year in years:                
            years_ob   = Select(year_dd).select_by_value(year[0])   #0 is most recent year (2023)
            leagues_ob = Select(league_dd).select_by_index(1) #starts at index 1
        
            # IF THINGS BREAK, UNCOMMENT THIS SECTION
            # Selects third dropdown menu, events
            # event_dd = self.driver.find_element(By.XPATH, '//select[@id="events"]')
            
            # Waits for third dropdown to populate with options based on leagues_ob                                        
            events_select = Select(event_dd)
            wait.until(lambda d: len(events_select.options) > 1)
            
            # Gets event options and ids for year and add to dictionary
            event_opts = Select(event_dd).options
            events = [opt.text for opt in event_opts[1:]]
            event_ids = [opt.get_attribute("value").split('/')[-1] for opt in event_opts[1:]]
            self.year_events[year[0]] = [(event, id) for event, id in zip(events, event_ids)]
            
            print(f'{year[0]}: {self.check_for_event_results(self.driver, event_dd, events)}')
    
    def check_for_event_results(self, driver, dd, events):
        """
        Iterate through each event and check if results exist
        input:
            N/A
        output:
            unsure yet
        """
        event_has_results = []
        for event in events:
            # Select event in events dropdown
            events_ob = Select(dd).select_by_visible_text(event)

            # Select fourth dropdown menu, categories
            cat_dd = driver.find_element(By.XPATH, '//select[@id="categories"]')
            cat_select = Select(cat_dd)
            wait = WebDriverWait(driver, 2)
            try:
                wait.until(lambda d: len(cat_select.options) > 1)
                event_has_results.append(1)
            except:
                event_has_results.append(0)
        return f'{sum(event_has_results)} of {len(events)} events have results!'
    
    def get_year_list(self):
        try:
            self.load_page(IFSCScraper.url)            
            wait = WebDriverWait(self.driver, 5)
            
            # The data we are after resides within an iframe
            wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe.jch-lazyloaded")))
        except:
            print('Error loading page!')
            
        # Dropdown menus for each choice
        year_dd, league_dd, event_dd, cat_dd = self.get_dropdowns(self.driver)
        year_opts = Select(year_dd).options
        return [year.text for year in year_opts]
    
    def get_single_year(self, year = '2022'):
        """
        Fully scrape each event for a given year
        input:
            year (string) - year to be scraped
        output:
            List of tuples (category, title, date, dataframe) to 
            be passed into function that generates actual .csv file
        """
        try:
            self.load_page(IFSCScraper.url)            
            wait = WebDriverWait(self.driver, 8)
            
            # The data we are after resides within an iframe
            wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe.jch-lazyloaded")))
            print(f'Scraping {year}...')
        except:
            print('Error loading page!')        
        
        # Dropdown menus for each choice
        year_dd, league_dd, event_dd, cat_dd = self.get_dropdowns(self.driver)
        
        # Select given year and league
        year_ob   = Select(year_dd).select_by_visible_text(year)
        league_ob = Select(league_dd).select_by_index(1)
        
        # Get list of all events for the year
        all_events = self.get_events(self.driver, event_dd)
        
        # Loop through each event, scrape results, and generate .csv file
        dfs = []
        for i, event in enumerate(all_events):
            # Implement check to see if event has already been scraped
            if self.check_if_scraped(event) and all_events[i] != all_events[i-1]:
                print(f'--Already scraped {event}!')
                continue
            # Some of the events aren't actually events, but more qualification rounds, and 
            # they don't list the results correctly, which will cause errors. The common thread
            # is the naming of them.
            elif event.count('(') < 1:
                print(f'--Skipping {event}...')
                continue
            else:
                # Set this flag for special cases where two events share the same name (rare)
                same_event_name = True if all_events[i] == all_events[i-1] else False
                                    
                # Select event
                if same_event_name:
                    event_ob = Select(event_dd).select_by_index(i+1)
                else:
                    event_ob = Select(event_dd).select_by_visible_text(event)
                    
                category_select = Select(cat_dd)

                # Some events were cancelled or don't have results listed, check for it here
                try:
                    wait.until(lambda d: len(category_select.options) > 1)
                    if not self.check_if_scraped(event):
                        # self.add_to_scraped_file(event)
                        print(f'--Scraping {event}...')
                except:
                    print(f'--No data for {event}!')
                    continue

                # Get results for each category
                for cat in category_select.options[1:]:
                    cat_ob = Select(cat_dd).select_by_visible_text(cat.text) # selects category

                    # Finds table with desired data
                    try:
                        wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@id="table_id_wrapper"]')))
                    except:
                        print(f'----No data for {cat.text}!')
                        continue

                    table_wrapper = self.driver.find_element(By.XPATH, '//div[@id="table_id_wrapper"]')
                    results = table_wrapper.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')

                    # Get event name and date
                    event_details = self.driver.find_element(By.XPATH, '//div[@class="labels"]')
                    event_results = event_details.find_elements(By.TAG_NAME, 'p') # Event title & date

                    # Get filename to check if it exists already
                    file = self.generate_filename((cat.text, event_results[0].text, event_results[1].text))
                    text = '--' + file
                    path = self.get_dir(cat.text)
                    
                    if same_event_name:
                        text = text[:-4] + '2' + '.csv'
                        file = file[:-4] + '2' + '.csv'
                    
                    filepath = os.path.join(path, file)
                                        
                    # Checks if the filename has been added to the .txt, AND if the file exists
                    if self.check_if_scraped(text, filepath):
                        continue
                    else:
                        print(f'----Scraping {cat.text}...')
                        
                        # Data (list of dictionaries) contains each climber's results
                        data = []
                        for result in results:
                            # Each climber's result stored in dict
                            temp_dict = self.scrape_results(event, result, cat.text)
                            
                            if temp_dict:
                                data.append(temp_dict)
                            else:
                                print(f'----Data format error for {cat.text}!')

                        # Create dataframe after collecting all the data
                        df = pd.DataFrame.from_dict(data)
                        
                        # Convert raw results into a .csv and marks file as scraped
                        self.convert_to_csv(file, cat.text, df)
                        self.add_to_scraped_file('--' + file)
                        
                # All categories for the event have been scraped
                self.add_to_scraped_file(event)
                        
    def get_athlete_height(self, athlete_id):
        """
        Scrape athlete page for climber's height
        input:
            athlete_id (int) - unique id assigned to each climber
        output:
            Height (in cm)
        """
        try:
            self.load_page(IFSCScraper.url2 + athlete_id, athlete_page=1)            
        except:
            print('Error loading page!')
            self.driver.quit()
        
        height_div = self.driver.find_element(By.XPATH, '//div[@class="athlete-info left-side"]')
        height_ele = height_div.find_elements(By.TAG_NAME, 'div')[1]
        return height_ele.text.split(': ')[1]
    
    def add_to_scraped_file(self, text):
        if not self.check_if_scraped(text):
            with open(ALREADY_SCRAPED, 'a') as file:
                file.write(f'{text}\n')
                return
    
    def check_if_scraped(self, text, file = ''):
        with open(ALREADY_SCRAPED, 'r') as f:
            done = [x.strip() for x in f.readlines()]
            
        if file:
            if text in done and os.path.exists(file):
                return True
            return False
        return text in done

    def scrape_results(self, event, result, cat):
        details = result.find_elements(By.TAG_NAME, 'td')
        athlete_id = details[1].find_element(By.TAG_NAME, 'a').get_attribute('href').split('id=')[1]
        
        if "LEAD" in cat or "BOULDER" in cat:
            try:
                temp_dict = {
                    "Event": event,
                    "ID": athlete_id,
                    "Rank": details[0].text,
                    "Name": f"{details[1].text} {details[2].text}",
                    "Country": details[3].text,
                    "Qualification": details[4].text,
                    "Semi-Final": details[5].text,
                    "Final": details[6].text
                }
            except:
                return False
        elif "SPEED" in cat:
            try:
                temp_dict = {
                    "Event": event,
                    "ID": athlete_id,
                    "Rank": details[0].text,
                    "Name": f"{details[1].text} {details[2].text}",
                    "Country": details[3].text,
                    "Qualification": details[4].text,
                    "Final": details[5].text
                }
            except:
                return False
        else:
            try:
                temp_dict = {
                    "Event": event,
                    "ID": athlete_id,
                    "Rank": details[0].text,
                    "Name": f"{details[1].text} {details[2].text}",
                    "Country": details[3].text,
                    "Qualification": details[4].text
                }
            except:
                return False
        return temp_dict
    
    def get_dropdowns(self, driver):
        year_dd   = driver.find_element(By.XPATH, '//select[@id="years"]')
        league_dd = driver.find_element(By.XPATH, '//select[@id="indexes"]')
        event_dd  = driver.find_element(By.XPATH, '//select[@id="events"]')
        cat_dd    = driver.find_element(By.XPATH, '//select[@id="categories"]')        
        return year_dd, league_dd, event_dd, cat_dd
    
    def get_events(self, driver, events_dd):
        event_opts = Select(events_dd)
        wait = WebDriverWait(driver, 10)
        wait.until(lambda d: len(event_opts.options) > 1)                    
        return [x.text for x in event_opts.options[1:]]
    
    def generate_filename(self, packed_data):
        # Unpacks data
        (category, event, date) = packed_data

        # Create filename in form of {date}_{event}_{category}
        date = ' '.join(date.split()[::-1][:2])       

        # Cleans up event name for next part
        event = event.replace('- ', '').split()
        if event[-1] == 'CANCELLED':
            event = ' '.join(event[:-2])
        else:
            event = ' '.join(event[:-1])

        # Uses Regex to clean because not every name has the same format
        filename = ' '.join([date, event, category])
        filename = re.findall("^[^\(]+|[\(].*", filename)
        filename[1] = filename[1].split(') ', 1)[1]
        filename = (''.join(filename)
                    .replace('(','[')
                    .replace(')',']')
                    .replace(' ', '_')
                    .replace(',', '')
                    .lower()) + '.csv'        
        return filename

    def convert_to_csv(self, filename, category, data):
        # Figure out correct directory
        path = self.get_dir(category)
        file = path + f'\\{filename}'

        # Generates .csv with filename
        data.to_csv(file, index=False)
            
    def get_dir(self, category):
        base = category.upper().split()
        if "MEN" in base:
            if "BOULDER" in base: return BOULDER_MEN_DIR
            if "LEAD" in base: return LEAD_MEN_DIR
            if "SPEED" in base: return SPEED_MEN_DIR
            if "COMBINED" in base: return COMBINED_MEN_DIR
            if "BOULDER&LEAD" in base: return BOULDER_AND_LEAD_MEN_DIR
        if "WOMEN" in base:
            if "BOULDER" in base: return BOULDER_WOMEN_DIR
            if "LEAD" in base: return LEAD_WOMEN_DIR
            if "SPEED" in base: return SPEED_WOMEN_DIR
            if "COMBINED" in base: return COMBINED_WOMEN_DIR
            if "BOULDER&LEAD" in base: return BOULDER_AND_LEAD_WOMEN_DIR
                
    def end_session(self):
        self.driver.quit()
    
    def scrape_site(self):
        self.get_years_events(self.get_comp_years_and_league())
        self.end_session()
        
    def scrape_all_ifsc_world_cups(self):
        years = self.get_year_list()
        for year in years[1:17]: # 2022-2007, 2023 events haven't happened yet
            scraper.get_single_year(year)
        self.end_session()
        
def display_events(dict):
        for k in dict:
            print(f'{int(k)}:')
            for v in dict[k]:
                print('    ', v)

In [167]:
# Takes ~37 minutes to scrape all years and events
scraper = IFSCScraper()
# scraper.scrape_all_ifsc_world_cups()
scraper.get_single_year('2022')
# scraper.end_session()

# scraper.get_athlete_height('1929')

Scraping 2022...
--Already scraped IFSC - Climbing World Cup (B) - Meiringen (SUI) 2022!
--Already scraped IFSC - Climbing World Cup (B,S) - Seoul (KOR) 2022!
--Already scraped IFSC - Climbing World Cup (B,S) - Salt Lake City (USA) 2022!
--Already scraped IFSC - Climbing World Cup (B) - Brixen (ITA) 2022!
--Already scraped IFSC - Climbing World Cup (B,L) - Innsbruck (AUT) 2022!
--Already scraped IFSC - Climbing World Cup (L,S) - Villars (SUI) 2022!
--Already scraped IFSC - Climbing World Cup (L,S) - Chamonix (FRA) 2022!
--Already scraped IFSC - Climbing World Cup (L) - Briançon (FRA) 2022!
--Already scraped IFSC - Climbing World Cup (L) - Koper (SLO) 2022!
--Already scraped IFSC - Climbing World Cup (L,S) - Edinburgh (GBR) 2022!
--Already scraped IFSC - Climbing World Cup (L,S) - Jakarta (INA) 2022!
--Already scraped IFSC - Climbing World Cup (B&L) - Morioka, Iwate (JPN) 2022!


# Analysis

In [5]:
def clean_boulder(df):
    # Gets tops and zones for qualifications
    df[['Q_Top', 'Q_Zone']] = df['Qualification'].str.split('T', expand=True)
    df['Q_Zone'] = df['Q_Zone'].str.split('Z', expand=True)[0]
    df[['Qualification', 'Q_Top_Att', 'Q_Zone_Att']] = df['Qualification'].str.split(expand=True)

    # Gets tops and zones for semi-finals
    df[['S_Top', 'S_Zone']] = df['Semi-Final'].str.split('T', expand=True)
    df['S_Zone'] = df['S_Zone'].str.split('Z', expand=True)[0]
    df[['Semi-Final', 'S_Top_Att', 'S_Zone_Att']] = df['Semi-Final'].str.split(expand=True)

    # Gets tops and zones for finals
    df[['F_Top', 'F_Zone']] = df['Final'].str.split('T', expand=True)
    df['F_Zone'] = df['F_Zone'].str.split('Z', expand=True)[0]
    df[['Final', 'F_Top_Att', 'F_Zone_Att']] = df['Final'].str.split(expand=True)

    new_cols = ['Q_Top', 'Q_Zone', 'Q_Top_Att', 'Q_Zone_Att',
                'S_Top', 'S_Zone', 'S_Top_Att', 'S_Zone_Att',
                'F_Top', 'F_Zone', 'F_Top_Att', 'F_Zone_Att']

    # Convert all new columns to int
    for col in new_cols:
        df[col] = df[col].astype('float', errors = 'ignore')

    return df

In [159]:
# Grabs all .csv files in given directory
all_files = glob.glob(os.path.join(BOULDER_MEN_DIR, "*.csv"))
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df['Rank'] = df['Rank'].fillna(-1).astype('int')
df

,Event,ID,Rank,Name,Country,Qualification,Semi-Final,Final
0,IFSC Climbing Worldcup (B) - Hall (AUT) 2007,1204,1,KILIAN FISCHHUBER,AUT,6T11 6B9,4T11 4B8,4T8 4B5
1,IFSC Climbing Worldcup (B) - Hall (AUT) 2007,3577,2,JORG VERHOEVEN,NED,5T9 6B10,3T5 4B4,4T9 4B5
2,IFSC Climbing Worldcup (B) - Hall (AUT) 2007,4783,3,AKITO MATSUSHIMA,JPN,3T4 6B15,3T10 3B3,2T7 2B4
3,IFSC Climbing Worldcup (B) - Hall (AUT) 2007,1213,4,DAVID LAMA,AUT,4T7 6B9,3T7 4B8,2T8 4B10
4,IFSC Climbing Worldcup (B) - Hall (AUT) 2007,849,5,GABRIELE MORONI,ITA,4T11 5B11,3T5 4B5,1T3 3B4
...,...,...,...,...,...,...,...,...
7786,"IFSC - Climbing World Cup (B,S) - Seoul (KOR) ...",3665,48,LUKE GOH WEN BIN,SGP,0T1Z 0 2,NaN,NaN
7787,"IFSC - Climbing World Cup (B,S) - Seoul (KOR) ...",11859,48,DOHYEON KIM,KOR,0T1Z 0 2,NaN,NaN
7788,"IFSC - Climbing World Cup (B,S) - Seoul (KOR) ...",2120,51,SUNGHOON PARK,KOR,0T1Z 0 5,NaN,NaN
7789,"IFSC - Climbing World Cup (B,S) - Seoul (KOR) ...",13548,52,MUHAMMAD FERZA FERNADA ABDI,INA,0T0Z 0 0,NaN,NaN


In [160]:
# Function to correct the old scoring syntax
def convert_score(score):
    if score:
        tmp = str(score)
        if 'b' in tmp.lower():
            tops, top_att = tmp.lower().split()[0].split('t')
            zones, zone_att = tmp.lower().split()[1].split('b')

            new_score = tops + 'T' + zones + 'Z ' + top_att + ' ' + zone_att
            return new_score
        return score
    return score

# Applies above function to the appropriate columns
df['Qualification'] = df['Qualification'].apply(convert_score)
df['Semi-Final'] = df['Semi-Final'].apply(convert_score)
df['Final'] = df['Final'].apply(convert_score)
df

,Event,ID,Rank,Name,Country,Qualification,Semi-Final,Final
0,IFSC Climbing Worldcup (B) - Hall (AUT) 2007,1204,1,KILIAN FISCHHUBER,AUT,6T6Z 11 9,4T4Z 11 8,4T4Z 8 5
1,IFSC Climbing Worldcup (B) - Hall (AUT) 2007,3577,2,JORG VERHOEVEN,NED,5T6Z 9 10,3T4Z 5 4,4T4Z 9 5
2,IFSC Climbing Worldcup (B) - Hall (AUT) 2007,4783,3,AKITO MATSUSHIMA,JPN,3T6Z 4 15,3T3Z 10 3,2T2Z 7 4
3,IFSC Climbing Worldcup (B) - Hall (AUT) 2007,1213,4,DAVID LAMA,AUT,4T6Z 7 9,3T4Z 7 8,2T4Z 8 10
4,IFSC Climbing Worldcup (B) - Hall (AUT) 2007,849,5,GABRIELE MORONI,ITA,4T5Z 11 11,3T4Z 5 5,1T3Z 3 4
...,...,...,...,...,...,...,...,...
7786,"IFSC - Climbing World Cup (B,S) - Seoul (KOR) ...",3665,48,LUKE GOH WEN BIN,SGP,0T1Z 0 2,NaN,NaN
7787,"IFSC - Climbing World Cup (B,S) - Seoul (KOR) ...",11859,48,DOHYEON KIM,KOR,0T1Z 0 2,NaN,NaN
7788,"IFSC - Climbing World Cup (B,S) - Seoul (KOR) ...",2120,51,SUNGHOON PARK,KOR,0T1Z 0 5,NaN,NaN
7789,"IFSC - Climbing World Cup (B,S) - Seoul (KOR) ...",13548,52,MUHAMMAD FERZA FERNADA ABDI,INA,0T0Z 0 0,NaN,NaN


In [154]:
# name = 'KOKORO FUJII'
# # Gets all rows for Tomoa
# df.groupby(df['Name']).get_group(name)

# # # Gets number of time Tomoa appeared for each round
# # df.groupby(df['Name']).get_group(name).count()['Qualification']
# # df.groupby(df['Name']).get_group(name).count()['Semi-Final']
# # df.groupby(df['Name']).get_group(name).count()['Final']

## Climber Statistics

In [161]:
# Create a new dataframe with athlete's ID, name, and country
comp_appearances = df.drop_duplicates(subset=['Name', 'ID'])[['ID', 'Name', 'Country']]

# Counts number of appearances per round per climber
qual_all = df.groupby(['ID', 'Name'])['Qualification'].count()
semi_app = df.groupby(['ID', 'Name'])['Semi-Final'].count()
final_app = df.groupby(['ID', 'Name'])['Final'].count()

# Merges the above dataframes
rounds = pd.merge(pd.merge(qual_all, semi_app , left_on=['ID', 'Name'], right_index=True), final_app, left_on=['ID', 'Name'], right_index=True)
comp_appearances = pd.merge(comp_appearances, rounds, left_on=['ID', 'Name'], right_index=True)

# Calculates percentages of appearances in each round
comp_appearances['Q_Pct'] = round((comp_appearances['Qualification'] / comp_appearances['Qualification']) * 100, 2)
comp_appearances['S_Pct'] = round((comp_appearances['Semi-Final'] / comp_appearances['Qualification']) * 100, 2)
comp_appearances['F_Pct'] = round((comp_appearances['Final'] / comp_appearances['Qualification']) * 100, 2)
comp_appearances = comp_appearances.sort_values('Final', ascending=False).reset_index(drop=True)
comp_appearances.head()

,ID,Name,Country,Qualification,Semi-Final,Final,Q_Pct,S_Pct,F_Pct
0,1204,KILIAN FISCHHUBER,AUT,56,51,44,100.0,91.07,78.57
1,53,DMITRII SHARAFUTDINOV,RUS,65,53,40,100.0,81.54,61.54
2,60,RUSTAM GELMANOV,RUS,69,56,38,100.0,81.16,55.07
3,2272,KOKORO FUJII,JPN,54,48,27,100.0,88.89,50.00
4,79,ALEKSEY RUBTSOV,RUS,57,50,26,100.0,87.72,45.61


In [162]:
# Convert 'ID' to string so we can scrape site for heights (~33 minutes)
comp_appearances['ID'] = comp_appearances['ID'].apply(str)
comp_appearances['Height'] = comp_appearances['ID'].apply(lambda x: scraper.get_athlete_height(x))

# Saves data so we don't have to scrape later
file = DATA_DIR + '\\climber_stats.csv'
comp_appearances.to_csv(file)

In [164]:
# Number of climbers that have a height listed on the IFSC site
(comp_appearances['Height'].str.contains('-')).sum()

1149

## Country Statistics

In [168]:
country_appearances = pd.DataFrame()
country_appearances['Country'] = comp_appearances['Country'].unique()

# Counts number of appearances per round per country
qual_all = df.groupby(['Country'])['Qualification'].count()
semi_app = df.groupby(['Country'])['Semi-Final'].count()
final_app = df.groupby(['Country'])['Final'].count()

# Merges the above with our previous dataframe
country_appearances = pd.merge(pd.merge(qual_all, semi_app , left_on='Country', right_index=True),
                               final_app, left_on='Country', right_index=True)

# Calculates percentages of appearances vs total events
country_appearances = country_appearances.sort_values('Final', ascending=False).reset_index()
country_appearances['Q_Pct'] = round((country_appearances['Qualification'] / country_appearances['Qualification'].sum()) * 100, 2)
country_appearances['S_Pct'] = round((country_appearances['Semi-Final'] / country_appearances['Semi-Final'].sum()) * 100, 2)
country_appearances['F_Pct'] = round((country_appearances['Final'] / country_appearances['Final'].sum()) * 100, 2)
country_appearances.head(20)

,Country,Qualification,Semi-Final,Final,Q_Pct,S_Pct,F_Pct
0,JPN,651,386,134,8.36,18.15,20.43
1,RUS,425,197,109,5.46,9.26,16.62
2,FRA,628,299,81,8.06,14.06,12.35
3,AUT,390,142,75,5.01,6.68,11.43
4,GER,495,180,38,6.35,8.46,5.79
5,SLO,294,131,34,3.77,6.16,5.18
6,KOR,156,58,29,2.00,2.73,4.42
7,CZE,173,49,26,2.22,2.30,3.96
8,CAN,333,65,24,4.27,3.06,3.66
9,ITA,345,100,19,4.43,4.70,2.90


In [169]:
df[df['Country'] == 'JPN']['Name'].unique()

array(['AKITO MATSUSHIMA', 'KAZUMA WATANABE', 'HIROSHI OKANO',
       'YUKI SUZUKI', 'KEITA MOGAKI', 'MASATOSHI SUGITA',
       'DAISUKE KONISHI', 'TSUKURU HORI', 'NOBUYUKI NAGATA',
       'TOSHIAKI TAKEUCHI', 'TATSUYA MURAOKA', 'SACHI AMMA',
       'ATSUSHI SHIMIZU', 'REI SUGIMOTO', 'SHINTA OZAWA',
       'HIROTARO HOSHINA', 'KEITA SETO', 'FUMIHIRO OYAMA', 'HIDEKAZU ITO',
       'IMASHI HASHIMOTO', 'SHINPEI HASEGAWA', 'RIMU MURAKAMI',
       'SHIGEO IWAI', 'AKITO TSUMORI', 'RYUICHI MURAI', 'MAKOTO YAMAUCHI',
       'YOSUKE SENBONGI', 'TATSUMI NITTA', 'TOMOAKI TAKATA',
       'MINORU NAKANO', 'KOKORO FUJII', 'YOSHIYUKI OGATA',
       'TOMOA NARASAKI', 'MASAHIRO HIGUCHI', 'KEITA WATABE', 'YUJI INOUE',
       'KAI HARADA', 'MEICHI NARASAKI', 'YUTA KOBATA', 'TOMOKI MUSHA',
       'NAOKI SHIMATANI', 'YUKI HADA', 'RYOHEI KAMEYAMA',
       'HAYATO NAKAMURA', 'JUN SHIBANUMA', 'TAISEI ISHIMATSU',
       'KAITO WATANABE', 'YUTA IMAIZUMI', 'HIBIKI YAMAUCHI', 'YUYA KITAE',
       'TAITO NAKAGAMI'

In [170]:
cols = ['Qualification', 'Semi-Final', 'Final', 'Q_Pct', 'S_Pct', 'F_Pct']
for col in cols:
    print(country_appearances[col].sum())

7791
2127
656
99.97999999999999
100.01000000000002
99.97000000000001
